# 🖼️ 16x16 Pixel Art Items LoRA - Gradio Interface

Before running ensure to change the runtime to a T4 GPU.

To run, simply press the play button on the left side of each code block. The process can take 5-10 minutes to install everything.


After running the last block, a gradio link will appear which will take you to the interface to generate your art.

In [ ]:
!nvidia-smi #Should show GPU info, if you're on T4 GPU

## Dependencies/SetUp

In [ ]:
!pip install --upgrade gradio
import gradio as gr

In [ ]:
# !git clone https://github.com/huggingface/diffusers
# !pip install /content/diffusers
!pip install diffusers
from diffusers import StableDiffusionPipeline
import torch

In [ ]:
!accelerate config default --mixed_precision 'fp16'

In [ ]:
!wget https://github.com/Jhr-4/PixelArt_LoRA/raw/main/16x16_lora.safetensors

## Load Base Model: SD v1.5

In [ ]:
model_name = 'stable-diffusion-v1-5/stable-diffusion-v1-5'

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_name, torch_dtype=torch.float16)
pipe.to("cuda")

## Gradio Setup

In [ ]:
from PIL import Image
pipe.safety_checker = None #16x16 pixel outputs were getting flagged incorrectly.

def generate(prompt, steps, guidance):
  if not prompt.strip():
    return None, None

  LoRA_gallery = []
  base_gallery = []

  pipe.load_lora_weights("/content/16x16_lora.safetensors")

  generated = pipe(f"min3craft, {prompt}", num_inference_steps=steps, guidance_scale=guidance, height=256, width=256, num_images_per_prompt=4).images #min3craft is style trigger
  for i in generated:
      pixelated = i.resize((16,16), Image.NEAREST)
      scaledPreview = pixelated.resize((256,256), Image.NEAREST)
      LoRA_gallery.append(scaledPreview)



  pipe.unload_lora_weights()

  generated = pipe(f"min3craft, {prompt}", num_inference_steps=steps, guidance_scale=guidance, height=256, width=256, num_images_per_prompt=4).images
  for i in generated:
      pixelated = i.resize((16,16), Image.NEAREST)
      scaledPreview = pixelated.resize((256,256), Image.NEAREST)
      base_gallery.append(scaledPreview)


  return LoRA_gallery, base_gallery

In [ ]:
with gr.Blocks() as interface:

  gr.Markdown("## 🖼️ 16x16 Pixel Art Items LoRA")
  with gr.Accordion("✨ | Additional Info & Tips", open=False):
    gr.Markdown("""
      Generate by entering a propmt and clicking generate. Items can be easily isolated by removing the background.
      For best results, specify the object, color, and cateogry in a comma-separated list. Additionally, adjust the steps and guidance scale.
      Example: "diamond helmet, blue, armor"

      - **Steps:** The iterations the model takes to produce the output (Lower = faster, but noisey; Higher = slower, but potentially higher quality)
      - **Guidance Scale:** Controls how strongly the prompt is followed (Lower = prompt can be ignored; Higher = less creativity)

      **Challenges/Limitations:**
      - Due to the small size of the dataset, some items won't turn out well
      - White items may not generate well due to training on white backgrounds
      - Current use case is limited to 2d item generation like game assets
      """)

  with gr.Row():
    with gr.Column():
      LoRA_gallery = gr.Gallery(label="LoRA Images", interactive=False)
    with gr.Column():
      base_gallery = gr.Gallery(label="Base SD 1.5 Images", interactive=False)

  with gr.Row():
    with gr.Column(scale=1):
      prompt = gr.Textbox(label="Prompt", placeholder="e.g., chair, wooden")
      generate_button = gr.Button("Generate", variant="primary")

    with gr.Column():
      steps = gr.Slider(minimum=1, maximum=100, step=1, value=50, label="Steps")
      guidance = gr.Slider(minimum=1, maximum=20, step=0.1, value=7.5, label="Guidance Scale")


  generate_button.click(
    fn=generate,
    inputs=[prompt, steps, guidance],
    outputs=[LoRA_gallery, base_gallery],
    queue=True
    )

interface.launch()